In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm     

***Cargamos el dataframe obtenido en la etapa de pre-procesamieno***

In [2]:
def load_data(file):
    
    df = pd.read_csv(file)
    df.drop(columns = ['Unnamed: 0'], inplace = True)
    
    return df
df = load_data('dataframe_limpio2323.csv')

***Definimos las variables X e Y***

In [3]:
X = df[['por_consenso_federal_paso', 'por_fit_paso',
       'por_MAS_paso', 'por_Frente_todos_paso', 'por_frente_patriota_paso',
       'por_juntos_cambio_paso', 'por_accion_vecinal_paso',
       'por_autonomista_paso', 'por_unite_paso', 'por_NOS_paso',
       'por_votos_blanco_paso']]

y = df[['por_consenso_federal_gen', 'por_fit_gen', 'por_juntos_cambio_gen',
       'por_frente_todos_gen', 'por_NOS_gen', 'por_Unite_gen',
       'por_blancos_gen']]

***Creamos una función para obtener las regresiones***

In [4]:
r2 = []

def get_ols_regression(y,column_name,X):
    model = sm.OLS(y[column_name], X).fit()
    r_squared = model.rsquared
    r2.append(r_squared)
    model_summary = model.summary()
    results_as_html = model_summary.tables[1].as_html()
    table = pd.read_html(results_as_html, header=0, index_col=0)[0]
    table.reset_index(inplace = True)
    table.rename(columns = {'coef':column_name, 'index': 'Features'}, inplace = True)
    return table

In [5]:
results_JxC = get_ols_regression(y, 'por_juntos_cambio_gen', X)
results_frente_todos = get_ols_regression(y, 'por_frente_todos_gen', X)
results_consenso_federal = get_ols_regression(y, 'por_consenso_federal_gen', X)
results_unite = get_ols_regression(y, 'por_Unite_gen', X)
results_fit = get_ols_regression(y, 'por_fit_gen', X)
results_blancos = get_ols_regression(y, 'por_blancos_gen', X)
results_nos = get_ols_regression(y, 'por_NOS_gen', X)

In [6]:
r2

[0.9012713289770744,
 0.8971813479842055,
 0.4685221401646211,
 0.2101543774035468,
 0.20418830224213202,
 0.14583136783601514,
 0.4193946344645316]

***Creamos la matriz de transferencia***

In [7]:
def features_repetidas():
    features = ['Consenso Federal', 'FIT', 'MAS',
       'Frente de Todos', 'Frente Patriota', 'Juntos por el Cambio',
       'Acción Vecinal', 'Partido Autonomista', 'Unite',
       'Frente Nos', 'Votos en Blanco']
    
    total_votos_paso = [df['Votos_consenso_federal_paso'].sum(), df['Votos_fit_paso'].sum(),
                        df['Votos_MAS_paso'].sum(), df['Votos_Frente_todos_paso'].sum(),
                        df['Votos_frente_patriota_paso'].sum(), df['Votos_juntos_cambio_paso'].sum(), df['Votos_accion_vecinal_paso'].sum(),
                       df['Votos_autonomista_paso'].sum(), df['Votos_unite_paso'].sum(), df['Votos_NOS_paso'].sum(), df['votos_blanco_paso'].sum()]
    
    features = pd.DataFrame(features, columns = ['Features'])
    total_votos_paso = pd.DataFrame(total_votos_paso , columns = ['Total_votos_paso'])
    
    features_votos_paso = features.join(total_votos_paso)
    features_votos_paso = pd.concat([features_votos_paso]*7 , ignore_index = True)
    
    features_votos_paso.reset_index(drop = True , inplace = True)
    
    features_votos_paso['Coefficients'] = pd.Series(dtype=int)
    
    features_votos_paso.reset_index(drop = True , inplace=True)
    return features_votos_paso

def y_repetidas():
    
    ys_rep = ['Juntos por el Cambio',
       'Frente de Todos', 'Consenso Federal', 'Unite',
       'FIT', 'Frente Nos','Votos en blanco']
    
    total_votos_gen = [df['Votos_juntos_cambio_gen'].sum(), df['Votos_Frente_todos_gen'].sum(),
                        df['Votos_consenso_federal_gen'].sum(), df['Votos_unite_gen'].sum(),
                        df['Votos_fit_gen'].sum(), df['Votos_NOS_gen'].sum(), df['votos_blanco_gen'].sum()]
    

    ys_rep = pd.Series(ys_rep)
    ys_rep = ys_rep.repeat(11)
    
    total_votos_gen = pd.Series(total_votos_gen)
    total_votos_gen = total_votos_gen.repeat(11)
    
    ys_rep = pd.DataFrame(ys_rep , columns = ['ys_rep'])
    ys_rep.reset_index(drop = True, inplace = True)
    
    total_votos_gen = pd.DataFrame(total_votos_gen , columns = ['Total_votos_gen'])
    total_votos_gen.reset_index(drop = True, inplace = True)
    
    
    ys_total = ys_rep.join(total_votos_gen)
    
    ys_total = ys_total.reset_index(drop = True)
    
    return ys_total

features_votos_paso = features_repetidas()
ys_total = y_repetidas()

results_table = features_votos_paso.join(ys_total)
results_table.ys_rep.unique()


array(['Juntos por el Cambio', 'Frente de Todos', 'Consenso Federal',
       'Unite', 'FIT', 'Frente Nos', 'Votos en blanco'], dtype=object)

In [8]:
def insert_coefficients(slice1, slice2, df, column_name):
    
    results_table.loc[slice1 : slice2, 'Coefficients'] = df[column_name].values.reshape(-1,1)

    return results_table

results_table = insert_coefficients(0,10, results_JxC, 'por_juntos_cambio_gen')
results_table = insert_coefficients(11,21, results_frente_todos, 'por_frente_todos_gen')
results_table = insert_coefficients(22,32, results_consenso_federal, 'por_consenso_federal_gen')
results_table = insert_coefficients(33,43, results_unite, 'por_Unite_gen')
results_table = insert_coefficients(44,54, results_fit, 'por_fit_gen')
results_table = insert_coefficients(55,65, results_nos, 'por_NOS_gen')
results_table = insert_coefficients(66,76, results_blancos, 'por_blancos_gen')

In [21]:
results_table

,Features,Total_votos_paso,Coefficients,ys_rep,Total_votos_gen
0,Consenso Federal,1251064.0,0.3762,Juntos por el Cambio,6748101.0
1,FIT,365498.0,0.2507,Juntos por el Cambio,6748101.0
2,MAS,97762.0,0.0054,Juntos por el Cambio,6748101.0
3,Frente de Todos,6653143.0,0.0145,Juntos por el Cambio,6748101.0
4,Frente Patriota,34598.0,0.2165,Juntos por el Cambio,6748101.0
...,...,...,...,...,...
72,Acción Vecinal,29203.0,0.1112,Votos en blanco,197306.0
73,Partido Autonomista,22435.0,-0.0384,Votos en blanco,197306.0
74,Unite,345802.0,-0.0408,Votos en blanco,197306.0
75,Frente Nos,450918.0,-0.0053,Votos en blanco,197306.0


In [22]:
results_table.to_csv(r'C:\Users\Administrador\Desktop\Carrera de Data Science\DS-Cor\Transferencia votos\dataframe_tabla_resultados_transferncia.csv')